Kyunghyun Cho, Bart van Merrienboer, Caglar Gulcehre, Dzmitry Bahdanau, Fethi Bougares, Holger Schwenk, Yoshua Bengio (2014), "Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation" 논문을 기반으로 합니다.

In [102]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy

import random
import math
import os
import time

SEED = 1

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

## 해당 아키텍처에 대한 논의
- 저번에 다뤘던 "Sequence to Sequence Learning with Neural Networks" 논문의 아키텍처에서는 Encoder와 Decoder의 개념에 대해서 이야기하고, Machine traslation 을 맥락을 잡아보았습니다.
- 학습을 통해, 번역의 성능이 향상된다는 점에서 의미가 있었지만, 해당 논문의 아키텍처에는 한계점이 존재하였는데, 바로 `information compression` 이라는 부분입니다. 
- Encoder 가 반환하는 hidden_state 또는 context vector ($h_{t}$) 는 Decoder 의 initial hidden state($s_{0}$)가 되었는데 여기서 context vector는 source sequence data에 대한 정보를 하나의 state로 압축시킨 형태가 됩니다.
- 이에 따라서, 모든 sequence data에 대한 정보가 흐려지게 되고, 이에 대한 한계가 대두되어, 이어서 Seq2Seq 기반의 논문들이 나오게 되는데, 그 중 첫 번째 논문이 바로 해당 논문  "Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation" 이 되겠습니다.
- 수식적으로 표현하면 아래와 같습니다.
    - Sequence to Sequence Learning with Neural Networks : 
        - $h_{t} = EncoderRNN(x_{t},h_{t-1})$ : encoder에는 source data 와 hidden state가 필요, hidden_state를 반환
        - $s_{0} = z = h_{T}$ : decoder의 initial hidden state는 encoder의 마지막 hidden_state
        - $s_{t} = DecoderRNN(y_{t},s_{t-1})$ : encoder와 같은 decoder의 network architecture
    - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation : 
        - $s_{t} = DecoderRNN(y_{t},s_{t-1},z)$ : decoder에서 context vector input 이 추가됩니다.
        - $y_{t+1}(pred) = f(y_{t},s_{t},z) $ : $y_{t}$ 를 가지고 fully_connected layer를 해줌으로써, $y_{t+1}$를 예측하는 데, 이 때 linear layer에는 $t$ 시점의 predicted_output(trg data), hidden_state, context_vector 가 변수로 들어갑니다. 

<img src = 'RNN_seq2seq_2.png'>

- 이제 본격적으로 아키텍처에 대해서 알아볼까요?
    - 위의 식을 계속 보시면서 따라오면 편하신데, 새롭게 짜여진 아키텍처에서는 y_t가 update되면서, context vector인 z가 계속 함수 안에 들어가게 됩니다. 즉, 기존에는 decoder의 initial hidden state로 사용되던 context vector가 이제 전체 시퀀스에 동일하게 들어가게 되는 것이죠.
    

In [103]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<eos>', lower=True)
TRG = Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<eos>', lower=True)

### `<sos>` 와 `<eos>` 에 대한 논의
- 아시는 분도 계시겠지만, 각각 Start Of Sentence, End of Sentence 를 의미합니다.
- Encoder와 Decoder 에 들어가는 source data, target data에 대해 앞뒤로 해당 두 토큰이 들어가게 됩니다.
- 문장의 시작과 끝을 일괄적으로 적용시켜줌으로써, 문장의 시작점과 끝점을 알려주는 역할을 합니다.
- CNN에서 이미지 처리를 할 때, 모서리 부분에 padding을 적용해줌으로써, 이미지의 경계를 학습시키는 효과와 유사합니다.

In [104]:
train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(SRC, TRG))
# 영어와, 독일 데이터를 다운받으면서, train, validation, test 데이터로 나눠서 가져오게 됩니다.

In [105]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)
# 최소한 2번 이상 나오는 vocab에 대해서만, numericalize 시키게 됩니다.

In [106]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 1000

train_iter, test_iter = BucketIterator.splits\
    (
    (train_data, test_data), batch_size=BATCH_SIZE, device=device
    )

# device 는 cpu 또는 gpu 를 적용하게 되고, (pytorch에서는 Variable(device=device) 와 같은 형태로, 
#인풋 데이터에 대해서 device 를 할당해줍니다.
# batch_size 를 할당해주면, 반환값에 randomly batch가 적용됩니다.(1000개씩 묶인 상태에서, 인덱스가 랜덤으로 섞인 iterator 가 됩니다.)

### Encoder
- 분류 모델과 내부는 크게 다르지 않습니다. (many-to-one)
- 마지막 결과값은 outputs 값이 아닌, hidden_layer 값을 받는 다는 것에 특징이 있습니다.
    - 마지막 outputs 값은 반환하는 hidden 값과 같습니다.
- encder 가 주는 hidden 값을 decoder 가 받아야 하기 때문에 약속되어야(assert?) 되어야 하는 것이 있습니다.
    - 인코더와 디코더의 layer 층의 갯수는 같아야 합니다.(모델에서는 num_layers로 선언되어 있습니다.)
    - 인코더와 디코더의 hidden_layer의 차원(dimension)이 같아야 합니다. (모델에서는 hidden_dim으로 선언되어 있습니다.)

In [107]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, num_layers, dropout, batch_size):
        
        super().__init__()     
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(input_dim, embedding_dim,padding_idx=1)
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers=num_layers, 
                            dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.hidden_dim = hidden_dim    
        
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        outputs, hidden = self.rnn(embedded)
        # rnn 의 argument 에서 embedding_vector 만 넣고, initial_hidden이나 initial_cell을 따로 넣어주지 않게 되면, 
        # dimension 이 자동으로, 맞춰져서 0벡터로 들어가게 됩니다.
        # hidden 은 가변 길이의 문장을 하나의 정보로 압축시키는 context vector 라고 생각하면 됩니다.
        # 최적의 context vector는 training 과정에서 최적화되는 파라미터입니다.
        # 이러한 convext vector는 num_layer 의 갯수만큼 있고, decoder part에서 풀게 됩니다.
        return hidden
    # hidden 변수 안에는 GRU의 경우 cell state가 없습니다. [1,batch_size, hidden_dim]
    # LSTM 의 경우에는 tuple 형태로 ([1,batch_size, hidden_dim],[1,batch_size, hidden_dim])
    

In [108]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 1
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
BATCH_SIZE = 1000
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT,BATCH_SIZE)

In [109]:
batch = next(iter(train_iter))
testing_hidden = enc(batch.src)
testing_hidden.size()
# [1,batch_size, hidden_dim]

torch.Size([1, 1000, 512])

### Decoder
- 해당 논문의 Decoder 의 input 값과 output 값을 이해함을 통해 네트워크를 살펴보도록 하겠습니다.
- Decoder 는 `세 가지 인풋값`과, `한 가지 반환값`을 가지는데, 각자의 특성에 대해 이야기해보겠습니다.
- INput data : 
    - inputs : Encoder 의 경우에서는 한번에 sequential data를 넣어줍니다. ['I','am','a','boy']를 한번에 넣어준다는 것이죠. 정보를 압축시키는 인코더의 역할적 맥락상 당연한 듯 합니다. 하지만, Decoder의 경우에는, training 과정에서는 target data(True value)를 넣어주지 않기 때문에, 출력해야 하는 문장의 길이가 얼마가 되어야 하고, (['나','는','소년','입니다'] 와 같은..) 얼마나 잘 맞추고 있는지에 대한, 기준점이 될만한 것이 필요합니다. 따라서, 단어를 하나씩 읽어드린다는 느낌으로, 데이터를 하나씩 넣고 예측 단어가 가질 수 있는 모든 경우의 수를 늘여놓은 벡터를 반환하게 되는 것이죠. 말이 좀 길어졌지만, 아래 그림의 오른쪽의 파랑색 부분의 네트워크를 보시면 좀 더 이해가 쉬우실 것 같습니다.
    - hidden : 처음엔 Encoder의 마지막 hidden state $h_t$ 가 같고, step을 진행함에 따라, update되는 변수입니다.        
    - context :  Encoder의 마지막 hidden state $h_t$ 를 의미합니다.
- OUTput data :
    - outputs : rnn 베이스 네트워크는 input 데이터가 hidden layer를 거쳐서 output 예측 데이터를 반환하게 되는데, many-to-many 네트워크는 문장을 반환해야 하고, 하나의 output이 반환할 수 있는 경우의 수는 모든 target data가 가지고 있는 vocab의 수가 됩니다. 따라서, `[hidden_layer's dimension, num of target_data's vocab]`에 대한 linear 함수를 씌워주어, 하나의 input 당  num of target_data's vocab만큼의 길이를 가지는 벡터를 반환하게 됩니다. 

### 이번 아키텍처에 대한 보다 깊은 논의

<img src ='rnn_seq2seq.png'>

- 이번에는 outputs의 형태보다는, 이러한 outputs이 어떻게 형성되었는지에 대해서 이야기하는 것이 더욱 중요할 것 같습니다.
- context vector를 모든 sequence step에 어떻게 일괄적으로 넣어줄 수 있을까요?


1. 처음에는 Embedding vector에 context vetor를 concatenate 시켜줍니다.
    - Embedding vector's dim : [1,batch,embedding_dim] 
    - Context vector's dim : [1,batch,hidden_dim]
    - concated_embedding vector's dim : [1,batch, embedding_dim + hidden_dim]

2. 이러한 임베딩 벡터를 RNN 네트워크에 넣고 나온 output , hidden

3.  $y_{t+1}(pred) = f(y_{t},s_{t},z) $ 에 따라, outputs, hidden, context vector를 넣고 fully_connect하는 것이 최종 outputs

In [110]:
class Decoder(nn.Module):
    def __init__(self, output_dim, embedding_dim, hidden_dim, num_layers, dropout):
        
        super().__init__()        
        self.embedding = nn.Embedding(output_dim, embedding_dim,padding_idx=1)
        self.hidden_dim = hidden_dim # decoder 의 hidden_dim와 같아야 합니다.
        self.output_dim = output_dim # 나올 수 있는 target data vocab의 모든 경우의 수 길이가 됩니다.
        self.num_layers = num_layers # decoder 의 num_layers와 같아야 합니다.
        self.linear = nn.Linear(embedding_dim + hidden_dim*2, output_dim)
        self.rnn = nn.GRU(embedding_dim+hidden_dim, hidden_dim, num_layers=num_layers, dropout=dropout)
        # hidden_dim *2 인 이유는, hidden, cell 두 개가 concat 되었는데, 이러한 hidden을 fully_connected 시켜서 입니다.
        self.dropout = nn.Dropout(dropout)        
        
    def forward(self, inputs, hidden, context):
        
        # hidden 은 초기에는 context 와 같지만, 나중에는 update되는 변수입니다.
        # context는 계속해서 고정적으로 embedding vector에 들어가는 변수입니다.
        
        inputs = inputs.unsqueeze(0) 
        # input은 train_iter 또는 test_iter의 이터레이터에 있는, trg 가 됩니다. 
       
        embedded = self.dropout(self.embedding(inputs))
        concated_embedded = torch.cat((embedded,context),dim=2)        
        output, hidden = self.rnn(concated_embedded,hidden)
        
        #output : 
        # 원래대로면 dimension 이 [문장의 길이, batch_size , hidden_dim * n_direction] 인데,
        # 여기서 sent_length 가 무조건 1이 됩니다. 왜냐면 context vector에서 압축된 정보를 input 으로 받아오기 때문입니다.
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        # hidden , cell의 dimension은 같다. 
        output = torch.cat((embedded.squeeze(0), hidden.squeeze(0), context.squeeze(0)), 
                           dim = 1)
        
        outputs = self.linear(output.squeeze(0))
        #prediction : output.squeeze_(0)'s dimension // [batch_size , hidden_dim]
        # prediction's dimension : [batch_size , output_dim]
        
        return outputs , hidden

In [111]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 1
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
BATCH_SIZE = 1000
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT,BATCH_SIZE)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

In [112]:
testing_hidden.size()

torch.Size([1, 1000, 512])

In [113]:
dec(batch.trg[0],testing_hidden,testing_hidden)

(tensor([[ 0.0775, -0.0566,  0.5983,  ..., -0.2491,  0.5507, -0.3622],
         [ 0.2004, -0.1633,  0.4167,  ...,  0.8854,  0.2569, -1.1642],
         [ 0.1979, -0.2888,  0.0685,  ...,  0.4897,  0.2731,  0.0900],
         ...,
         [ 0.2770, -0.3386, -0.0887,  ...,  0.4116,  0.5397, -1.0487],
         [-0.3438, -0.1755,  0.1574,  ...,  0.6431,  0.6543, -0.6588],
         [-0.4971,  0.4489,  0.5461,  ...,  0.4226,  0.2081, -0.1657]],
        grad_fn=<AddmmBackward>),
 tensor([[[ 0.0132, -0.2921,  0.0892,  ..., -0.2381, -0.0025, -0.3103],
          [-0.0766, -0.3606, -0.3623,  ..., -0.3297,  0.1986, -0.0025],
          [-0.3166, -0.3405,  0.1986,  ..., -0.3240, -0.0667, -0.3141],
          ...,
          [-0.0831, -0.3876, -0.3821,  ...,  0.0165, -0.1277, -0.1536],
          [-0.1792, -0.2685,  0.2522,  ..., -0.2239, -0.0652, -0.1337],
          [ 0.2795, -0.3687, -0.0797,  ..., -0.3359,  0.3422, -0.2655]]],
        grad_fn=<StackBackward>))

### Seq2Seq 클래스에 있는 teacher_forcing_ratio에 대한 논의
- 보다 자세한 설명은 https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/ 을 참고하세요
- RNN 베이스 뉴럴 네트워크의 대표적 단점 중 하나인 느린 수렴 속도를 극복하기 위해서, 디코더의 인풋 데이터 일부에 실제 target data를 넣어주는 것입니다. 

In [114]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # decoder 의 맨처음에는 encoder 에서 나온 hidden,cell을 넣어주어야 합니다! 이때, num_layer와 hidden_dim은 같아야 합니다!
        hidden = self.encoder(src)
        context = hidden.clone()
        # decoder 를 돌면서, 각 단어에 대한 outputs값(벡터 형태)이 나오게 되는데, 이러한 값들을 아래의 outputs 변수에 저장해줍니다
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        # 맨 처음에는 문장의 시작을 알리는 sos(start of sentence) 토큰을 넣어주어야 합니다.
        input_ = trg[0,:]
        for t in range(1, max_len):
            # for 문이 돈다는 것은, many-to-many의 네트워크가 한 칸씩 옆으로 이동한다는 뜻과 같습니다.
            output, hidden = self.decoder(input_, hidden, context)
#            output'dimension : [batch_size , output_dim], 여기서 output_dim 은 출현 가능한 모든 target lang 의 수 입니다.
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1] # 해당 글자의 numericalized index 를 넣어주어야 합니다.
            input_ = (trg[t] if teacher_force else top1)
            # teacher_forcing 을 할 경우, 실제 trg데이터를 다음 input으로 사용, 그렇지 않을 경우, 이전 state에서 가장 높은 
            # 값을 가진[나올 수 있는 모든 target vocab 리스트 중에서를 의미합니다. 확률값의 형태는 아니지만, 가장 개연성이 높은 단어를 의미합니다.]
        return outputs

In [115]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 1
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
BATCH_SIZE = 1000
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT,BATCH_SIZE)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [116]:
batch = next(iter(train_iter))

In [117]:
model(batch.src,batch.trg).size()

torch.Size([33, 1000, 5893])

### gradient clipping 에 대한 논의
- https://hskimim.github.io/Avoid_Exploding_gradient_in_Neural_Net_with_gradient_clipping/ 에 보다 자세히 기록하였습니다.
- RNN 베이스의 네트워크의 특징인 gradient exploding 을 방지해주는 방법론입니다.

In [118]:
class fit() : 
    
    def __init__(self, model, train_iter, test_iter, epoch = 5) : 
        self.optimizer = optim.Adam(model.parameters())
        # <pad> 토큰은 임베딩 벡터와, loss_function에 argument 로 들어가서, training 과정에서 제외됩니다.
        self.pad_idx = TRG.vocab.stoi['<pad>'] 
        self.criterion = nn.CrossEntropyLoss(ignore_index=self.pad_idx)
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(device)
        self.train_iter = train_iter
        self.test_iter = test_iter
        self.epoch = epoch
            
    def train(self,clip):
    
        epoch_loss = 0 # loss per epoch
        self.model.train()
        
        for i, batch in enumerate(self.train_iter):
            print('batch : ',i,end='\r')
            src = batch.src
            trg = batch.trg

            self.optimizer.zero_grad()

            output = model(src, trg)        

            loss_output = output[1:].view(-1, output.shape[-1])
            loss_trg = trg[1:].view(-1)
            # sos 토큰을 제외하고, 차원을 맞춘 후에, output을 변수에 저장해줍니다.
            
            loss = self.criterion(loss_output, loss_trg)
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            # gradient clipping
            self.optimizer.step()
            
            epoch_loss += loss.item()
        
        return epoch_loss / len(self.train_iter)
    

    def fit_by_iterate(self,clip) : 
        
        for epoch in range(self.epoch):
            print('epoch : ',epoch + 1)
            train_loss= self.train(clip)
            print("epoch's loss : {}".format(train_loss))

In [119]:
fitting_process = fit(model,train_iter,test_iter,epoch=5)

In [120]:
fitting_process.fit_by_iterate(1)

epoch :  1


KeyboardInterrupt: 